In [ ]:
import pandas as pd
from utils import *

In [ ]:
# -- PRODUCT_FORECAST doeltabel --

# PRODUCT_FORECAST_ID
# PRODUCT_FORECAST_expected_volume
# INVENTORY_LEVELS_DATA_inventory_count
# PRODUCT_production_cost
# PRODUCT_margin
# YEAR_MONTH_year
# YEAR_MONTH_month
# PRODUCT_product_number